#### Problem Description : Sales forecasting of a pharmaceutical retail chain


KioMed, a huge pharmaceutical retailer, operates in in 10+ cities across India. The
company has one distribution warehouse in each of the cities it operates in.
Unfortunately, the warehouses are not able to consistently meet the demand of the
stores in their respective cities. Kio, the parent company, being a data driven
corporation wants to solve the stocking / inventory management problem using their
in-house data science team.

The retailer has provided you with historical sales data and is looking to forecast the
sales for the period of one month after the end of the data. These forecasts will be
used to ensure that the company is able to stock its supplies of medicines in a
warehouse accordingly in each city for a period of one month. 

The company will also provide you with the footfall data for all the stores across each
of its cities. You can use this data, but as in the case of the real world, the footfall data
is only available at train time and not at test time


### Importing  only necessary Libraries for merging all the csv files

In [1]:
import numpy as np
import pandas as pd
import json
import math
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
path=os.getcwd() # getting the location of current file
path

'C:\\Users\\jayam\\Downloads\\7879phd'

#### Let us work with one file at a time to save memory

In [3]:
train=pd.read_csv(path +"\\train_data.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])# reading train data

In [4]:
train.shape # shape of train data

(22624345, 6)

##### Create date column 

In [5]:
train["date"]=train.apply(lambda row:str(int(row["year"]))+"-"+str(int(row["month"]))+"-"+str(int(row["day"])),axis=1)

In [6]:
train["date"]=pd.to_datetime(train["date"])## converting to date type

##### Checking null values

In [7]:
train.isnull().sum()

year        0
month       0
day         0
city        0
medicine    0
sales       0
date        0
dtype: int64

In [8]:
train.head(2) # see head of the data

,year,month,day,city,medicine,sales,date
0,2015,1,2,1,1,24.0,2015-01-02
1,2015,1,2,1,2,144.0,2015-01-02


In [9]:
train.tail(2) # see tail of data

,year,month,day,city,medicine,sales,date
22624343,2018,6,30,10,3157,4.0,2018-06-30
22624344,2018,6,30,10,3165,56.0,2018-06-30


##### Checking duplicates in the experiment file and  we have drawn conclusions about 1827 duplicates in train data 

In [10]:
#trainRowsDF = train[train.duplicated()]
#trainRowsDF.shape = (1827, 7)

#####  We could see that there are 2 sale prices for a particular a given city and medicine on a given day,so let us group and take the sum of sales and we could see from experiment that these are actually not duplicates

In [11]:
train=train.groupby(["year","month","day","city","medicine","date"])["sales"].sum().reset_index()

In [12]:
train.shape# shape after groupby

(22601741, 7)

In [13]:
discount=pd.read_csv(path +"\\discount_features.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])
discount.shape #read the file and see the shape 

(17277949, 4)

In [14]:
discount.head(2)# see head of the data

,date,city,medicine,discounted
0,2016-04-01,1,1,0
1,2016-04-01,1,2,0


In [15]:
discount.tail(2)# see tail of the data

,date,city,medicine,discounted
17277947,2018-07-31,10,3165,0
17277948,2018-07-31,10,3057,0


#### Checking null values

In [16]:
discount.isnull().sum()

date          0
city          0
medicine      0
discounted    0
dtype: int64

In [17]:
discount["date"]=pd.to_datetime(discount["date"])# converting to date format

In [18]:
discount["year"]=discount["date"].dt.year# extracting year
discount["month"]=discount["date"].dt.month# extracting month
discount["day"]=discount["date"].dt.day# extracting day

In [19]:
discount["discounted"].value_counts()

0    16478900
1      799049
Name: discounted, dtype: int64

#### Checking duplicates

In [20]:
discount[discount.duplicated()].shape

(22009, 7)

#### As discussed in train dataset same thing applies here too in case of duplicates where as in discounts after taking sum let us take replace 2 by 1 as it is a catagorical column

In [23]:
discount=discount.groupby(["year","month","day","city","medicine","date"])["discounted"].sum().reset_index()

In [24]:
discount.shape# shape after groupby

(17255345, 7)

In [25]:
discount["discounted"].replace(2,1,inplace=True)

In [26]:
discount["discounted"].value_counts()

0    16456732
1      798613
Name: discounted, dtype: int64

#### Let us merge both train and discount data files

In [27]:
train=train.merge(discount,on=["year","month","day","city","medicine","date"],how="left")

In [28]:
train.shape ## shape of merged file

(22601741, 8)

In [29]:
train.isnull().sum() ## checking nulls after merging

year                0
month               0
day                 0
city                0
medicine            0
date                0
sales               0
discounted    6119931
dtype: int64

#### We could see null values in discounted column as the data for discounts is available from 2016 April 

####  So, either we can drop the rows or we can assume that there is no discount provided for these rows 

In [30]:
footfall=pd.read_csv(path +"\\foot_fall.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])# read footfall data

In [31]:
footfall.shape# footfall shape

(10, 1268)

In [32]:
footfall.isnull().sum()# null values in footfall

city          0
2015-01-02    0
2015-01-03    0
2015-01-04    0
2015-01-05    0
             ..
2018-06-26    0
2018-06-27    0
2018-06-28    0
2018-06-29    0
2018-06-30    0
Length: 1268, dtype: int64

In [33]:
footfall["2018-01-02"]## there are null values in jan 2nd 2018

0    13500.0
1    19868.0
2    21024.0
3    19360.0
4    18296.0
5        NaN
6        NaN
7    14676.0
8    18156.0
9        NaN
Name: 2018-01-02, dtype: float64

In [34]:
footfall["2017-12-31"]### for backward fill

0    15312.0
1    21184.0
2    20720.0
3    17608.0
4    17824.0
5    14408.0
6    17684.0
7    11792.0
8    21656.0
9    13504.0
Name: 2017-12-31, dtype: float64

In [35]:
footfall["2018-01-05"]### for forward fill

0    11868.0
1    16252.0
2    13608.0
3    14792.0
4    11736.0
5    11352.0
6    12228.0
7    10112.0
8    16452.0
9     7296.0
Name: 2018-01-05, dtype: float64

In [36]:
footfall["2018-01-02"].fillna(footfall["2018-01-05"],inplace=True)## filling nas with forwardfill

#### We could observe Jan 3rd and 4th are missing in footfall let us add them into the data

In [37]:
footfall["2018-01-03"]=footfall["2018-01-05"]
footfall["2018-01-04"]=footfall["2018-01-05"]

In [38]:
footfall.columns.nunique() ## finding number of unique columns

1270

In [39]:
footfall.head(2)## see head of footfall data

,city,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-10,...,2018-06-23,2018-06-24,2018-06-25,2018-06-26,2018-06-27,2018-06-28,2018-06-29,2018-06-30,2018-01-03,2018-01-04
0,Ahmedabad,9384.0,6808.0,8064.0,10248.0,11180.0,6876.0,6436.0,7104.0,5924.0,...,9584.0,11940.0,13040.0,12144.0,10164.0,9784.0,10472.0,10040.0,11868.0,11868.0
1,Bangalore,16644.0,14640.0,15660.0,19056.0,19740.0,14280.0,12740.0,13480.0,11928.0,...,11948.0,13860.0,16384.0,16192.0,12388.0,12124.0,13412.0,12172.0,16252.0,16252.0


In [40]:
footfall.tail(2)## see tail of footfall data

,city,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-10,...,2018-06-23,2018-06-24,2018-06-25,2018-06-26,2018-06-27,2018-06-28,2018-06-29,2018-06-30,2018-01-03,2018-01-04
8,Mumbai,19284.0,14472.0,16676.0,19684.0,19700.0,15484.0,14372.0,16024.0,13292.0,...,14088.0,16368.0,18800.0,18640.0,14772.0,14208.0,16044.0,14628.0,16452.0,16452.0
9,Other,8572.0,7180.0,6716.0,8616.0,9252.0,6852.0,6220.0,6808.0,5688.0,...,6164.0,6980.0,8056.0,8480.0,6480.0,6004.0,7144.0,6040.0,7296.0,7296.0


In [41]:
footfall=footfall.melt(id_vars=["city"],value_name="footfall",var_name="date")## lets change the configuration

In [42]:
footfall.head(2) ## see head after using melt function

,city,date,footfall
0,Ahmedabad,2015-01-02,9384.0
1,Bangalore,2015-01-02,16644.0


In [43]:
with open(path+"\\city_dict.json") as f: ## read json file and get cities in approriate format for mapping
      data = json.load(f)
city = {value: key for key, value in data.items()}
city={k:int(v) for k, v in city.items()}

In [44]:
footfall["city"]=footfall["city"].map(city)# map cities

In [45]:
footfall["date"]=pd.to_datetime(footfall["date"]) # convert to date format

In [46]:
train =pd.merge(train,footfall,on=["date","city"],how="left") # merge with train data

In [47]:
train.shape # check shape after merge

(22601741, 9)

In [48]:
train.isnull().sum() # check for nulls

year                0
month               0
day                 0
city                0
medicine            0
date                0
sales               0
discounted    6119931
footfall            0
dtype: int64

In [60]:
def get_year_month_str(row):
    year_month_str = str(int(row["year"])) + "_"
    month = int(row["month"])
    if month < 10:
        year_month_str = year_month_str + "0"
    year_month_str = year_month_str + str(month)
    return year_month_str

In [61]:
train["year_month"]=train.apply(get_year_month_str,axis=1)## creating city_medicine combination

In [53]:
train["city_medicine"]=train.apply(lambda row:str(int(row["city"]))+"_"+str(int(row["medicine"])),axis=1)## creating city_medicine combination

In [55]:
train["week"]=train["date"].dt.week ## create week number

In [63]:
train.to_csv("train_finaldata.csv",index=False)# create train  csv file for future reference

##### Reading test data set  and merging with other data sets

In [68]:
test=pd.read_csv(path +"\\test_data.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])# reading test data

In [73]:
test["date"]=test.apply(lambda row:str(int(row["year"]))+"-"+str(int(row["month"]))+"-"+str(int(row["day"])),axis=1)# creating date column

In [75]:
test["date"]=pd.to_datetime(test["date"])

In [76]:
test=test.merge(discount,on=["year","month","day","city","medicine","date"],how="left")

In [79]:
test =pd.merge(test,footfall,on=["date","city"],how="left")

In [81]:
test["year_month"]=test.apply(get_year_month_str,axis=1)## creating year_month combination

In [83]:
test["city_medicine"]=test.apply(lambda row:str(int(row["city"]))+"_"+str(int(row["medicine"])),axis=1)## creating city_medicine combination

In [84]:
test["week"]=test["date"].dt.week ## create week number

In [86]:
test.to_csv("test_finaldata.csv",index=False)# create test  csv file for future reference

In [87]:
footfall.to_csv("footfall_finaldata.csv",index=False)# create footfall  csv file for future reference

In [88]:
discount.to_csv("discount_finaldata.csv",index=False)# create discount  csv file for future reference

#### Lets create visulizations in another file